# Thinkful Data Science Course

#Unit 3: Answering Questions with Data

##Lesson 1: How New Yorkers Bike

In this exercise, the goal is to record the number of bikes available every minute for an hour across all of New York City in order to see which station or set of stations is the most active in New York City for that hour. Activity is defined as the total number of bicycles taken out or returned in an hour. So if 2 bikes are brought in and 4 bikes are taken out, that station has an activity level of 6.


In [1]:
# Requests is a package that allows us to download data from any online resource.
import requests
# Because it's in JSON format, you have to do something a little different to import it:
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import pandas as pd
# To use counter:
import collections
from statistics import median
import sqlite3 as lite
# a package with datetime objects
import time
# a package for parsing a string into a Python datetime object
from dateutil.parser import parse 
import datetime


Obtaining data

In [2]:
r = requests.get('http://www.citibikenyc.com/stations/json')

con = lite.connect('citi_bike.db')
cur = con.cursor()

In [3]:
r = requests.get('http://www.citibikenyc.com/stations/json')

con = lite.connect('citi_bike.db')
cur = con.cursor()

In [4]:
key_list = [] #unique list of keys for each station listing
for station in r.json()['stationBeanList']:
    for k in station.keys():
        if k not in key_list:
            key_list.append(k)

In [5]:
df = json_normalize(r.json()['stationBeanList'])


Here are the steps to modify your code to run for an hour, downloading the file every minute, and saving the result. The goal is determine the station with the most change in that hour. To start out, checkout the branch "repeat_query":

In [ ]:
with con:
    cur.execute('''CREATE TABLE citibike_reference (id INT PRIMARY KEY, totalDocks INT, city TEXT, altitude INT, stAddress2 TEXT, longitude NUMERIC, postalCode TEXT, testStation TEXT, stAddress1 TEXT, stationName TEXT, landMark TEXT, latitude NUMERIC, location TEXT )''')

In [ ]:
sql = '''INSERT INTO citibike_reference (
	id, 
	totalDocks, 
	city, 
	altitude, 
	stAddress2, 
	longitude, 
	postalCode, 
	testStation, 
	stAddress1, 
	stationName, 
	landMark, 
	latitude, 
	location) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)'''

In [ ]:
#for loop to populate values in the database
with con:
    for station in r.json()['stationBeanList']:
        #id, totalDocks, city, altitude, stAddress2, longitude, postalCode, testStation, stAddress1, stationName, landMark, latitude, location)
        cur.execute(sql,(station['id'],station['totalDocks'],station['city'],station['altitude'],station['stAddress2'],station['longitude'],station['postalCode'],station['testStation'],station['stAddress1'],station['stationName'],station['landMark'],station['latitude'],station['location']))

In [ ]:
# To get multiple readings by minute, the availablebikes table is going 
# to need to be a little different. In this case, the station ID (id) 
# is going to be the column name, but since the column name can't 
# start with a number, you'll need to put a character in front of the 
# number ("").

# With 332 stations, this is best done in code:
#extract the column from the DataFrame and put them into a list
station_ids = df['id'].tolist() 

#add the '_' to the station name and also add the data type for SQLite
station_ids = ['_' + str(x) + ' INT' for x in station_ids] 


In [ ]:
#create the table
#in this case, we're concatentating the string and joining all the station ids (now with '_' and 'INT' added)

with con:
	cur.execute("CREATE TABLE available_bikes ( execution_time INT, " +  ", ".join(station_ids) + ");")



for i in range(60):
    
    r = requests.get('http://www.citibikenyc.com/stations/json')
    exec_time = parse(r.json()['executionTime'])

    cur.execute('INSERT INTO available_bikes (execution_time) VALUES (?)', (exec_time.strftime('%s'),))
    con.commit()

    id_bikes = collections.defaultdict(int)
    for station in r.json()['stationBeanList']:
        id_bikes[station['id']] = station['availableBikes']

    for k, v in id_bikes.items():
        cur.execute("UPDATE available_bikes SET _" + str(k) + " = " + str(v) + " WHERE execution_time = " + exec_time.strftime('%s') + ";")
   	# for k, v in id_bikes.items():
    #     print("UPDATE available_bikes SET _" + str(k) + " = " + str(v) + " WHERE execution_time = " + exec_time.strftime('%s') + ";")
    

    con.commit()

    time.sleep(60)

con.close() #close the database connection when done


In [4]:
# ANALYZING THE RESULTS

# Reading in the data:
con = lite.connect('citi_bike.db')
cur = con.cursor()

df = pd.read_sql_query("SELECT * FROM available_bikes ORDER BY execution_time",con,index_col='execution_time')


In [6]:
df

,_72,_79,_82,_83,_116,_119,_120,_127,_128,_137,...,_3210,_3211,_3212,_3213,_3214,_3215,_3216,_3217,_3220,_3221
execution_time,,,,,,,,,,,,,,,,,,,,,
1440469739,16,3,13,52,9,16,5,31,9,4,...,0,0,0,0,0,0,0,0,0,0
1440469794,16,3,13,52,9,16,5,31,9,4,...,0,0,0,0,0,0,0,0,0,0
1440469861,16,3,13,52,10,16,5,31,9,5,...,0,0,0,0,0,0,0,0,0,0
1440469916,16,3,13,52,10,16,5,31,9,6,...,0,0,0,0,0,0,0,0,0,0
1440469971,16,3,13,52,10,16,5,31,9,6,...,0,0,0,0,0,0,0,0,0,0
1440470037,16,3,13,52,10,16,5,31,9,6,...,0,0,0,0,0,0,0,0,0,0
1440470092,15,2,13,52,10,16,5,31,9,6,...,0,0,0,0,0,0,0,0,0,0
1440470158,15,2,13,52,10,16,5,31,9,6,...,0,0,0,0,0,0,0,0,0,0
1440470213,15,2,13,52,9,16,5,31,9,6,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# First you need to process each column and calculate the change each minute:
hour_change = collections.defaultdict(int) #default zero, 
for col in df.columns:
    station_id = col[1:] #trim the "_"
    print(station_id)
    station_vals = df[col].tolist()
    print(station_vals)
    station_change = 0
    for k,v in enumerate(station_vals):
        if k < len(station_vals) - 1:
            station_change += abs(station_vals[k] - station_vals[k+1])
        #if k:
        	# station_change += abs(v - station_vals[k-1])
    hour_change[int(station_id)] = station_change #convert the station id back to integer

# The enumerate() function returns not only the item in the list but 
# also the index of the item. This allows us to find the value 
# (with index of k) just after it in sequence (k + 1). We run the loop 
# until k is equal to the index for the second to last element in the list.

72
[16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15]
79
[3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 3]
82
[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
83
[52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
116
[9, 9, 10, 10, 10, 10, 10, 10, 9, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 10, 10, 8, 8,

In [14]:
# At this point, the values are in the dictionary keyed on the station ID. 
# To find the winner:

# def keywithmaxval(d):
#     # create a list of the dict's keys and values; 
# 	v = list(d.values())
# 	k = list(d.keys())
#     # return the key with the max value
# 	return(k[v.index(max(v))])

def keywithmaxval(d): 
	return max(d,key=lambda k:d[k])
	# max d would just be max station code, whereas here we need the station key with the max change

In [15]:
# assign the max key to max_station
max_station = keywithmaxval(hour_change)

In [16]:
max_station

521

In [17]:
# From there, you query the reference table for the important 
# information about the most active station:

#query sqlite for reference information
cur.execute("SELECT id, stationname, latitude, longitude FROM citibike_reference WHERE id = ?", (max_station,))
data = cur.fetchone()
print("The most active station is station id %s at %s latitude: %s longitude: %s " % data)
print("With " + str(hour_change[max_station]) + " bicycles coming and going in the hour between " + datetime.datetime.fromtimestamp(int(df.index[0])).strftime('%Y-%m-%dT%H:%M:%S') + " and " + datetime.datetime.fromtimestamp(int(df.index[-1])).strftime('%Y-%m-%dT%H:%M:%S'))

# This should print out the result. 
# Note that this will just print out the first station in the list that 
# has the max value. You should visually inspect the data to make sure 
# this is the case:

The most active station is station id 521 at 8 Ave & W 31 St latitude: 40.750967348716 longitude: -73.994442075491 
With 136 bicycles coming and going in the hour between 2015-08-25T06:28:59 and 2015-08-25T07:29:01


In [ ]:
plt.bar(hour_change.keys(), hour_change.values())
plt.show()